# Protocol

**TO DO**
- Compare details to the paper
- testing 

A workflow for simulation assisted interpretation of lipid-like densities. This notebook details the steps required to run and analyse simulations to aid characterisation of lipid-like densities observed in e.g. cryo-EM structures. 

The protocol implements the [PyLipID analysis toolkit](https://github.com/wlsong/PyLipID) for calculation of lipid binding sites
and their associated kinetics. The pipeline also details how best to screen and rank PyLipID outputs, compare the relative kinetics of lipids at sites where densities are observed and refine lipid poses using atomistic simulations. Thus, the protocol serves to assess both the most likely identity of the lipid species accounting for a given density and the lipid fingerprint within a membrane environment.  

The [GROMACS  simulation software](https:www.gromacs.org) (version 2019) was used throughout however all versions > GROMACS 5 should also be compatible. 

#### The protocol code can be decomposed into numbered stages corresponding to:

1. Setting up and performing coarse-grained simulations
2. Testing PyLipID cut-off values
3. Selecting PyLipID input parameters and running PyLipID analysis
4. Screening PyLipID data
5. Ranking site lipids
6. Setting up atomistic simulations to refine lipid poses

Within each subsection the code split into a list of **'USER DEFINED VARIABLES'** and the associated **'CODE'** for that section.

While the protocol establishes inputs needed for coarse-grained or atomistic simulations (i.e. `.tpr` files for gromacs) the simulations should be run by the user. Simulations may be run simulations locally or offload to a high performance computing facility, cluster or cloud computing network. Sections corresponding to simulation productions runs are marked by **'PAUSE POINT'**. Once simulations have reached completion the pipeline can be resumed to analyse and process data.



## Citation
Please cite the following if elements of this protocol are used in research:
[**ADD CITATION **]()

In [ ]:
import os
import protocol
import protocol.system_setup
import protocol.process_structure as pps
import protocol.simulation as ps
import protocol.test_PyLipID_cutoffs as lip_test
import protocol.run_PyLipID as lip_run
import protocol.screen_PyLipID_data as lip_screen
import protocol.rank_sites as rs

Establish paths to be used throughout the protocol. 

In [ ]:
protocol_path=os.path.dirname(protocol.__file__)
path=protocol.system_setup.setup(protocol_path, save_dir)

# 1: Setting up coarse-grained simulations

Within this section an atomistic protein structure (`protein_AT_full`) is converted to coarse-grained (CG) resolution using the [MARTINI forcefield](http://cgmartini.nl/). The protein is embedded within a bilayer and solvated with water and ions. Each system is minimised and subject to two equilibration steps. An output `md.tpr` file for each CG replicate is produced. 

#### The following options are used to define the CG simulation setup:
- `protein_AT_full`: The atomistic `.pdb` file of the protein. Protein residues should not have missing atoms but larger segment deletions are permitted provided the elastic network can maintain segment interconnectivity. 
- `nprot`: Number of homomeric protein chains, for heteromers use `nprot=1`.
- `protein_shift`: The protein is embedded within the bilayer using [`insane.py`](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.5b00209). This variable can be used to shift the z axail protein position in the membrane to align the transmembrane regions within the bilayer (value can also be negative).
- `protein_rotate`: Rotate the protein position to align transmembrane regions within the bilayer (angle in 'x y z').
- `boxsize`: CG simulation box size in x,y,z (nm).
- `save_dir`: Name of the directory where simulation and analyses will be stored.
- `forcefield`: Define the [MARTINI CG](http://cgmartini.nl/) forcefield to use in simulations. Currently compatible with 'martini_v2.0', 'martini_v2.1' and 'martini_v2.2'. The [ElNeDyN](https://pubs.acs.org/doi/abs/10.1021/ct9002114) elastic network is applied.
- `membrane_composition`:  Define the bilayer composition in which to embed the protein. The membrane composition can be selected from a list of predefined membrane compositions (see Table below) or defined in a custom manner using `insane.py` syntax. An example of a custom bilayer composition is `'-u POPC:50 -u DOPC:50 -l POPE:30 -l CHOL:10 -l DOPE:60'` where `-u` denotes the upper leaflet and `-l` the lower leaflet species.  
- `CG_simulation_time`: Simulation time in microseconds ($\mu$s). 
- `replicates`: Number of simulation replicates.
- `stride`: Used to define the number of trajectory frames to skip during trajectory processing and running PyLipID analysis (in subsequent sections).
- `n_cores`: Number of CPU to use to run gromacs `mdrun` commands

#### Current predefined membrane compositions available:

|Predefined bilayer name | Bilayer composition (`insane.py` syntax)|
|:---|:---|
|'Simple'|`'-u POPC:70 -u CHOL:30 -l POPC:70 -l CHOL:30'`|
|'Plasma membrane'|`'-u POPC:20 -u DOPC:20 -u POPE:5 -u DOPE:5 -u DPSM:15 -u DPG3:10 -u CHOL:25 -l POPC:5 -l DOPC:5 -l POPE:20 -l DOPE:20 -l POPS:8 -l DOPS:7 -l POP2:10 -l CHOL:25'`|
|'ER membrane'|`'-u POPC:37 -u DOPC:37 -u POPE:8 -u DOPE:8 -u CHOL:10 -l POPC:15 -l DOPC:15 -l POPE:20 -l DOPE:20 -l POPS:10 -l POP2:10 -l CHOL:10'`|
|'Raft-like microdomain'|`'-u DPPC:27 -u DPPE:8 -u DPSM:15 -u DPG3:10 -u CHOL:40 -l DPPC:15 -l DPPE:35 -l DPPS:10 -l CHOL:40'`|
|'Gram neg. inner membrane'|`'-u POPE:67 -u POPG:23 -u CDL2:10 -l POPE:67 -l POPG:23 -u CDL2:10'`|
|'Gram neg. outer membrane'|`'-u PGIN:100 -l POPE:90 -l POPG:5 -l CDL2:5'`|




### Section 1: USER DEFINED VARIABLES 

In [ ]:
protein_AT_full='TMD.pdb'
nprot = 1 
protein_shift=0 
protein_rotate='0 90 0' 
boxsize='10,10,9' 

save_dir="Test" 

forcefield='martini_v2.2' 
membrane_composition='Simple' #Predefined bilayer composition                                           
#membrane_composition='-u POPC:100 -l POPC:30 -l POPE:70' # Custom bilayer composition
CG_simulation_time=15 
replicates=10 
stride=10 
n_cores=16 

### Section 1a: CODE
Process and orient the structure in preparation for coarse-graining and insertion into the membrane.

In [ ]:
protein_AT_full=pps.process_structure(protocol_path, protein_AT_full, protein_rotate)

#### Setup the CG simulations
The oriented protein is converted to CG resolution using [`martinize.py`](http://cgmartini.nl/index.php/tools2/proteins-and-bilayers/204-martinize). The CG protein is inserted into the bilayer (defined with `membrane_composition`). The system is then solvated with MARTINI water and neutralised with NaCl (~0.15 M).  

**IMPORTANT**: check you are happy with the orientation of the protein within the membrane - if not then adjust the `protein_rotate` and `protein_shift` variables. 

In [ ]:
python3_path, dssp_path = ps.get_py_paths(protocol_path)
bilayer=ps.bilayer_select(membrane_composition)

ps.system_setup(protocol_path, path)
ps.fetch_CG_itp(forcefield, path)
ps.top_header(forcefield, path)
ps.run_CG(protocol_path, protein_AT_full, protein_shift, bilayer, boxsize, replicates, python3_path, dssp_path, n_cores, path, CG_simulation_time)

### Pause point - run CG simulations using the generated md.tpr files
A `md.tpr` file is generated for each simulation replicate. These can be run using the GROMACS `mdrun` command:

`gmx mdrun -deffnm md`

We recommend off-loading the simulations to e.g. a high performance computing facility or cluster to improve simulation running times.  

Once simulations have finished running, ensure the data is returned to the same location as the `md.tpr` file for each replicate.

### Section 1b: CODE
Once CG simulations have completed they can be processed using GROMACS `trjconv`. The simulations are made whole across periodic boundary conditions and `stride` number of frames are skipped. 

In [ ]:
ps.trjconv_CG(protocol_path, stride, replicates, path)

# 2: Testing PyLipID cut-off values

The PyLipID analysis tool uses a double cut-off scheme to define lipid interactions with a protein. In short, a protein-lipid contact is initiated when a lipid comes within the lower cut-off of the protein and ends when the protein-lipid distance surpasses the upper cut-off. This accounts for transient fluctuations in the position of the lipid due to Brownian motion. 

A key metric in using this pipeline is the choice of double cut-off used to define lipid interactions. To aid this selection, the following section describes how to test a range of cut-off values and make appropriate lower and upper cut-off choices. For further details regarding cut-off selection see [this tutorial](https://pylipid.readthedocs.io/en/master/tutorials/1-Distance_cutoff_determination.html). 

#### The following options are used in cut-off testing:
- `lipid_atoms`: A list of lipid atom/bead names to consider. The default (`None`) will weight all lipid beads equally during testing. 
- `contact_frames`: For each residue-lipid interaction, plot the interaction only if a contact was formed over X number of frames where X=`contact_frames`.
- `distance_threshold`: For each residue-lipid interaction, plot the interaction only if the minimum protein-lipid distance comes within X nm (where X=`distance_threshold` for greater than the number of `contact_frames`.  
- `lower_cutoff`: List of lower cut-off values to test in the exhaustive cut-off screen. 
- `upper_cutoff`: List of upper cut-off values to test in the exhaustive cut-off screen.
- `timeunit`: Simulation timeunit to use in plots (`"us"` or `"ns"`). 

### Section 2: USER DEFINED VARIABLES 

In [ ]:
lipid_atoms = None 
contact_frames = 30  
distance_threshold = 0.65 

lower_cutoff = [0.4, 0.425, 0.45, 0.475, 0.5, 0.55] 
upper_cutoff = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9] 
timeunit = "us"

### Section 2: CODE
Generate a list of lipids to test cut-offs:

In [ ]:
lip_list=lip_test.get_lipids(bilayer)

For each lipid in `lip_list` the probability distribution of minimum protein-lipid distances is plotted. A pairwise exhaustive test of all lower and upper cut-off combinations (provided with `lower_cutoff` and `upper_cutoff`) is performed using PyLipID.

The output is a plot of interactions durations, number of detected binding sites and number of contacting residues for each cut-off pair. These can be used to guide selection of a dual-cut-off scheme to use when running the full PyLipID analysis in the following section. 


In [ ]:
traj=lip_test.load_traj(path)
for lipid in lip_list:
    print("\n Testing:", lipid)
    fig_dir=lip_test.set_lipid(path, lipid)
    distance_set = lip_test.compute_minimum_distance(traj, lipid, fig_dir, 1, lipid_atoms=lipid_atoms,
                                               contact_frames=contact_frames, distance_threshold=distance_threshold)
    lip_test.plot_PDF(distance_set, 1000, "{}/PyLipID_cutoff_test_{}/dist_distribut_{}.pdf".format(path, lipid, lipid), lipid)

    cutoff_list, trajfile_list, topfile_list = lip_test.exhaustive_search_setup(path, lower_cutoff, upper_cutoff, replicates)
    num_of_binding_sites, duration_avgs, num_of_contacting_residues = lip_test.test_cutoffs(
                                     cutoff_list, trajfile_list, topfile_list, lipid, lipid_atoms,
                                     nprot=nprot, stride=stride, save_dir="{}/PyLipID_cutoff_test_{}".format(path, lipid), timeunit=timeunit)
    lip_test.ex_data_process(path, lipid, num_of_binding_sites, duration_avgs, num_of_contacting_residues, cutoff_list)
    lip_test.graph(cutoff_list, [num_of_binding_sites[cutoffs] for cutoffs in cutoff_list],
          "num. of binding sites", lipid, f"{path}/PyLipID_cutoff_test_{lipid}/test_cutoff_num_of_bs_{lipid}.pdf")
    lip_test.graph(cutoff_list, [duration_avgs[cutoffs] for cutoffs in cutoff_list],
          f"Durations ({timeunit})", lipid, f"{path}/PyLipID_cutoff_test_{lipid}/test_cutoff_durations_{lipid}.pdf")
    lip_test.graph(cutoff_list, [num_of_contacting_residues[cutoffs] for cutoffs in cutoff_list],
          "num. of contacting residues", lipid,
          f"{path}/PyLipID_cutoff_test_{lipid}/test_cutoff_num_of_contacting_residues_{lipid}.pdf")

#### Selecting cut-off values
- The **lower cut-off** should encapsulate the first peak in the probability distribution plot and correspond to the cut-off at which there is an increase in interaction durations, number of binding sites and number of residues comprising each site. 
- The **upper cut-off** should fully capture the first lipid solvation shell, corresponding roughly to the position of the first trough in the probability distribution plot (between the first and second distribution peaks). In addition, the choice of upper cut-off may also be dependant on whether deviations are observed in the exhaustive pair-wise cut-off search. The upper cut-off should correspond to the first plateau in e.g. the interaction duration, after which increasing the upper cut-off makes little difference. Increasing the upper cut-off beyond the first plateau may further increase interaction metrics by capturing the second lipid solvation shell which should be avoided.  

# 3. Selecting PyLipID input parameters and running PyLipID analysis

In this section [PyLipID](https://github.com/wlsong/PyLipID) will be used to calculate the lipid interaction profiles for each lipid and calculate lipid binding sites. PyLipID uses a community analysis approach to obtain the binding sites for each lipid and the kinetics associated with lipid binding to each site (Occupancy, Residence Time etc). Top ranked or clustered lipid binding poses for each site can also be obtained. The methodological underpinnings of PyLipID can be found [here](https://www.biorxiv.org/content/10.1101/2021.07.14.452312v1) (**bioRxiv link**). 

It is advised to test a range of dual cut-off combinations (see above) before selecting a lower/upper cut-off to run the full PyLipID analysis. 

#### The following variables are used to run PyLipID:
- `cutoffs`: List of the lower and upper cut-off to use for analysing the CG simulations. Alternativly, a single-cut-off scheme can be achieved by using the same value for two cut-offs.
- `lipid_atoms`: A list of lipid atom/bead names to consider. The default (`None`) will weight all lipid beads equally during analysis.  
- `dt_traj`: The timestep of the trajectories. Setting this parameter is not necessary for standard trajectory formats (e.g. `xtc`, `trr`). This variable must be set when when trajectories are in a format with no timestep information. 
- `binding_site_size`: Define the minimum number of residues that can consisute a binding site. This can be used to screen out very small 'binding sites' which may only result from transient interactions. 
- `n_top_poses`: Number of top ranked representative bound poses to be written out for each binding site.
- `n_clusters`: PyLipID will cluster the bound lipid poses for each binding site. This can be done in a supervised format by providing the number of clusters to generate for each site. Alternatively, the default (`"auto"`) will use an unsupervised density based clustering algorithm to find possible clusters.   
- `save_pose_format`: File format the lipid poses are written in. Any [format supported](https://mdtraj.org/1.9.4/hdf5_format.html) by [`mdtraj`](https://www.mdtraj.org/1.9.5/index.html) is accepted. 
- `save_pose_traj`: Save all the bound poses in a trajectory for each binding site. The generated trajectories may use up disk space (up to a couple GB depending on your system).
- `save_pose_traj_format`: The format for the saved pose trajectories. Any [format supported](https://mdtraj.org/1.9.4/hdf5_format.html) by [`mdtraj`](https://www.mdtraj.org/1.9.5/index.html) is accepted. 
- `timeunit`: Time unit used for reporting the results (microsecond: `"us"`, nanosecond: `"ns"`). 
- `resi_offset`: Offset the residue index number in the calculated interactions. 
- `radii`: This setting can be used to define the Van der Waals radii of any non-standard atoms/beads. These are used when calculating the binding site surface areas. Radii are provided in the format of a python dictionary `{atom_name: radius}`. The Van der Waals radii of common atoms are automatically defined by [mdtraj](https://github.com/mdtraj/mdtraj/blob/master/mdtraj/geometry/sasa.py#L56). The radii of MARTINI 2.2 beads are included within PyLipID.
- `pdb_file_to_map`: If a pdb coordinate of the receptor is provided, a python script `"show_binding_site_info.py"` will be generated which maps the binding site information to the structure in PyMol. As PyMol cannot connect CG structures well, an atomistic structure of the receptor is needed for this setting.
- `fig_format`: Format for all PyLipID produced figures. All formats that are supported by `matplotlib.pyplot.savefig()` are permitted. 
- `num_cpus`: The number of CPUs to use when functions are using multiprocessing. By default (`None`), the functions will use up all the cpus available. This can use up all the memory in some cases.

### Section 3: USER DEFINED VARIABLES

In [1]:
cutoffs = [0.5, 0.7]  # [lower, upper]
lipid_atoms = None 
dt_traj = None 

binding_site_size = 4  
n_top_poses = 3    
n_clusters = "auto"  

save_pose_format = "gro"  
save_pose_traj = True 
save_pose_traj_format = "xtc" 

timeunit = "us" 
resi_offset = 0  

radii = None  

pdb_file_to_map = "TMD.pdb"   

fig_format = "pdf"  

num_cpus = None  

### Section 3: CODE

Run PyLipID analysis for each lipid. Outputs are located within the Interaction_*Lipid* directory. 

In [ ]:
trajfile_list, topfile_list=lip_run.get_trajectories(path, replicates)
for lipid in lip_list:
    lip_run.run_pylipid(trajfile_list, topfile_list, dt_traj, stride, lipid, lipid_atoms, cutoffs, nprot, binding_site_size,
        n_top_poses, n_clusters, save_dir, save_pose_format, save_pose_traj, save_pose_traj_format, timeunit, resi_offset,
         radii, pdb_file_to_map, fig_format, num_cpus)

# 4. Screening PyLipID data

This section involves post-processing of PyLipID outputs. The binding site kinetics (Residence times, Occupancy), surface areas and $\Delta$$k_{off}$ are plotted in ranked order. 

The $\Delta$$k_{off}$ is the difference between the site $k_{off}$ derived via bi-exponential fitting to the survival time correlation function of interactions durations and the $k_{off}$ derived via bootstrapping to the same data. If a site is well converged then these $k_{off}$s should be similar (i.e. $\Delta$$k_{off}$ ~ 0). Therefore the $\Delta$$k_{off}$ can be used to assess the quality of binding site kinetics in addition to reported $R^{2}$ values for sites. We note that poorly defined sites may also have very low surface areas however there is no expected correlation between higher surface areas with site residence times or occupancies. 

The ranked binding site output can be useful for assessing which sites are more/most relevant and which sites are ill defined. Exclude poor sites from future analysis. 

There are no user defined variables for this section. 

### Section 4: CODE

In [ ]:
for lipid in lip_list:
    data = lip_screen.get_data(path, lipid)
    if data is not None:
        lip_screen.plot_screen_data(data, path, lipid)

# 5. Ranking site lipids

After assessing the quality of the binding site data obtained using PyLipID the representative lipid poses can be compared with the structural densities. 

#### For a given density surrounding the membrane protein structure:
- Show the binding sites mapped onto the protein structure using the `show_binding_site_info.py` script outputted by PyLipID:

`pymol show_binding_site_info.py`

- Compare the binding site positions to that of the density and note the binding site index of the corresponding site.
- Check the representative top ranked/clustered lipid poses (outputted by PyLipID) for the binding site. Assess whether the lipid pose could account for the observed structural densities. 
- Repeat for all lipid types. 


The `BindingSite_ID_dict` is a python dictionary used to compare the residence times of different lipids bound at similar site locations around the protein. The dictionary keys are the lipids. The dictionary value for each lipid is a list of binding site indices. The order of binding site indices should correspond to a given site/density i.e. site residence times are compared in the listed order. 

For example: `BindingSite_ID_dict={'POPC':[1, 3, 2], 'POPE': [4, 1, 2]}` means that POPC binding site 1 and POPE site 4 correspond to the same location on the protein. Likewise POPC-site-3 will be compared to POPE-site-1 and POPC-site-2 will be compared to POPE-site-2. Thus a comparison of site residence times for different lipids can be used to assess the most likely identity of lipid bound at a site/density. 

If the lipid does not have a binding site which matches the corresponding location of the density then replace the binding site index with `"X"`. 


### Section 5: USER DEFINED VARIABLES

In [ ]:
BindingSite_ID_dict={"POPC": [1, 2 , 5, 3],   
           "POPE": [2, 3, 6, 1],               
           "CHOL":[1, 3, 5, "X"]}

### Section 5: CODE

In [ ]:
rank_data=rs.get_BSstat(path, BindingSite_ID_dict)
rs.plot_site_rank(path, BindingSite_ID_dict, rank_data)

# 6. Setting up atomistic simulations to refine lipid poses

The final section of the protocol details the refinement of lipid poses using atomistic simulations. The script requires an input frame from the CG simulations with the lipid of interest bound to a site. PyLipID outputs the replicate number and simulation time point that representative top ranked or clustered poses were obtained from. Thus, the frame from which a lipid pose was taken can be readily obtained using e.g.:

`gmx trjconv -f md_stride.xtc -s md.tpr -n index.ndx -o md_stride_`*`time`*`.gro -dump` *`time`*

Backmapping the CG frame to atomistic resolution is enabled through used of [CG2AT](https://github.com/owenvickery/cg2at). Links detailing the [usage](https://github.com/owenvickery/cg2at) and [methodological details](https://pubs.acs.org/doi/full/10.1021/acs.jctc.1c00295) of CG2AT are provided.   

#### The following variables are used to setup atomistic simulations:
- `input_CG_frame`: The input CG frame for backmapping to atomistic detail using CG2AT.
- `model_type`: CG2AT backmaps the protein conformation based on either the conformation in the structure (`'aligned'`) or the conformation in the CG frame (`'denovo'`). In extreme cases when the protein conformation in the CG frame differs substantially from the structure it may not be possible to obtain the `'aligned'` conformation. Use this variable to define which of the protein conformations from CG2AT to use as input for atomistic simulations. By default the lipid and solvent conformations are backmapped based on their positions in the CG frame. 
- `replicates_AT`: Number of atomistic simulation replicates.
- `AT_simulation_time`: Simulation time in nanoseconds.

 ### Section 6: USER DEFINED VARIABLES

In [ ]:
input_CG_frame="md_stride_t500.gro" 
model_type="aligned"
replicates_AT=1 
AT_simulation_time=100 

### Section 6: CODE

In [ ]:
ps.CG2AT(protocol_path, protein_AT_full, input_CG_frame, save_dir)
AT_path=ps.system_setup_AT(protocol_path, path, model_type)
ps.run_AT(AT_path, replicates_AT, protocol_path, AT_simulation_time)

### Pause point - run atomistic simulations using the generated md.tpr files
A md.tpr file is generated for each simulation replicate. These can be run using the GROMACS `mdrun` command:

`gmx mdrun -deffnm md`

We recommend off-loading the simulations to e.g. a high performance computing facility or cluster to improve simulation running times.  

Once complete, lipid poses in the atomistic simulation can be compared to lipid or lipid-like densities in the structure. Details of this comparison, and quantitative evaluation using [Q scores](https://www.nature.com/articles/s41592-020-0731-1) can be found within the protocol manuscript.    
